In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import binance_framework as bf
from collections import deque
import random

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

In [3]:
CURRENCIES = ['ETHUSDT','BTCUSDT', 'XRPUSDT','LTCUSDT']
SEQUENCE_LEN = 128
FUTURE_PERIOD_PREDICT = 5
RATIO_TO_PREDICT = 'ETHUSDT'
EPOCHS = 20
BATCH_SIZE = 128
NAME = 'SEQ-'+ str(SEQUENCE_LEN)+'-FUTPRED-'+str(FUTURE_PERIOD_PREDICT)+'-PRED-'+str(time.time())

In [4]:
#coins = bf.get_data_by_intervals(300,CURRENCIES,bf.INTERVAL_1MINUTE)

In [5]:
#coins.to_csv('all_1min_300.csv',index=False)

In [6]:
coins  = pd.read_csv('all_1min_300.csv')

In [7]:
coins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149701 entries, 0 to 149700
Data columns (total 48 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   ETHUSDT Open Time                     149701 non-null  int64  
 1   ETHUSDT Open                          149701 non-null  float64
 2   ETHUSDT High                          149701 non-null  float64
 3   ETHUSDT Low                           149701 non-null  float64
 4   ETHUSDT Close                         149701 non-null  float64
 5   ETHUSDT Volume                        149701 non-null  float64
 6   ETHUSDT Close Time                    149701 non-null  int64  
 7   ETHUSDT Quote asset volume            149701 non-null  float64
 8   ETHUSDT n_trades                      149701 non-null  int64  
 9   ETHUSDT Taker buy base asset volume   149701 non-null  float64
 10  ETHUSDT Taker buy quote asset volume  149701 non-null  float64
 11  

In [8]:
coins.head()

,ETHUSDT Open Time,ETHUSDT Open,ETHUSDT High,ETHUSDT Low,ETHUSDT Close,ETHUSDT Volume,ETHUSDT Close Time,ETHUSDT Quote asset volume,ETHUSDT n_trades,ETHUSDT Taker buy base asset volume,...,LTCUSDT High,LTCUSDT Low,LTCUSDT Close,LTCUSDT Volume,LTCUSDT Close Time,LTCUSDT Quote asset volume,LTCUSDT n_trades,LTCUSDT Taker buy base asset volume,LTCUSDT Taker buy quote asset volume,LTCUSDT Ignore
0,1574785080000,145.99,146.08,145.87,145.88,241.28503,1574785139999,35218.416693,84,49.68388,...,46.66,46.59,46.59,28.24642,1574785139999,1316.735587,20,9.04073,421.691594,0
1,1574785140000,145.89,145.96,145.84,145.96,40.85429,1574785199999,5959.466443,49,20.42048,...,46.60,46.58,46.58,80.79035,1574785199999,3763.360087,10,7.27921,339.211186,0
2,1574785200000,145.98,145.98,145.78,145.81,140.04726,1574785259999,20423.385381,68,35.84368,...,46.61,46.57,46.57,85.55753,1574785259999,3985.330867,7,18.02712,840.244063,0
3,1574785260000,145.81,145.82,145.63,145.74,305.43565,1574785319999,44519.270184,117,173.11928,...,46.56,46.49,46.51,325.49714,1574785319999,15139.718570,42,90.97910,4232.232356,0
4,1574785320000,145.74,145.81,145.70,145.77,86.54447,1574785379999,12612.838273,52,18.12635,...,46.52,46.49,46.52,62.84014,1574785379999,2922.416068,15,16.18125,752.621786,0


In [9]:
bf.show_time_skips(coins, CURRENCIES)

First time difference is:  60000


ETHUSDT Time difference:  3660000
starting at index:  107141
ending at index:    107142
total frames since previous timeskip:  107141


ETHUSDT Time difference:  32299
starting at index:  122056
ending at index:    122057
total frames since previous timeskip:  14915


ETHUSDT Time difference:  21327701
starting at index:  122057
ending at index:    122058
total frames since previous timeskip:  1


ETHUSDT Time difference:  46701
starting at index:  141728
ending at index:    141729
total frames since previous timeskip:  19671


ETHUSDT Time difference:  7753299
starting at index:  141729
ending at index:    141730
total frames since previous timeskip:  1
First time difference is:  60000


BTCUSDT Time difference:  3660000
starting at index:  107141
ending at index:    107142
total frames since previous timeskip:  107141


BTCUSDT Time difference:  32287
starting at index:  122056
ending at index:    122057
total frames since previous timeskip:  14915


In [10]:
coins = coins[0:107141]
len(coins)

107141

In [11]:
bf.show_time_skips(coins, CURRENCIES)

First time difference is:  60000
First time difference is:  60000
First time difference is:  60000
First time difference is:  60000


In [12]:
bf.keep_one_timestamp(coins,CURRENCIES)

In [13]:
#coins.drop([(CURRENCIES[0]+' Open Time'), (CURRENCIES[0]+' Ignore')], axis=1, inplace=True)

In [14]:
coins.head()

,ETHUSDT Open,ETHUSDT High,ETHUSDT Low,ETHUSDT Close,ETHUSDT Volume,ETHUSDT Quote asset volume,ETHUSDT n_trades,ETHUSDT Taker buy base asset volume,ETHUSDT Taker buy quote asset volume,ETHUSDT Ignore,...,LTCUSDT High,LTCUSDT Low,LTCUSDT Close,LTCUSDT Volume,LTCUSDT Quote asset volume,LTCUSDT n_trades,LTCUSDT Taker buy base asset volume,LTCUSDT Taker buy quote asset volume,LTCUSDT Ignore,Timestamp
0,145.99,146.08,145.87,145.88,241.28503,35218.416693,84,49.68388,7254.209112,0,...,46.66,46.59,46.59,28.24642,1316.735587,20,9.04073,421.691594,0,1574785139999
1,145.89,145.96,145.84,145.96,40.85429,5959.466443,49,20.42048,2978.871293,0,...,46.60,46.58,46.58,80.79035,3763.360087,10,7.27921,339.211186,0,1574785199999
2,145.98,145.98,145.78,145.81,140.04726,20423.385381,68,35.84368,5227.199591,0,...,46.61,46.57,46.57,85.55753,3985.330867,7,18.02712,840.244063,0,1574785259999
3,145.81,145.82,145.63,145.74,305.43565,44519.270184,117,173.11928,25234.657077,0,...,46.56,46.49,46.51,325.49714,15139.718570,42,90.97910,4232.232356,0,1574785319999
4,145.74,145.81,145.70,145.77,86.54447,12612.838273,52,18.12635,2641.885972,0,...,46.52,46.49,46.52,62.84014,2922.416068,15,16.18125,752.621786,0,1574785379999


In [15]:
coins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107141 entries, 0 to 107140
Data columns (total 41 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   ETHUSDT Open                          107141 non-null  float64
 1   ETHUSDT High                          107141 non-null  float64
 2   ETHUSDT Low                           107141 non-null  float64
 3   ETHUSDT Close                         107141 non-null  float64
 4   ETHUSDT Volume                        107141 non-null  float64
 5   ETHUSDT Quote asset volume            107141 non-null  float64
 6   ETHUSDT n_trades                      107141 non-null  int64  
 7   ETHUSDT Taker buy base asset volume   107141 non-null  float64
 8   ETHUSDT Taker buy quote asset volume  107141 non-null  float64
 9   ETHUSDT Ignore                        107141 non-null  int64  
 10  BTCUSDT Open                          107141 non-null  float64
 11  

In [16]:
coins.index[0]

0

In [17]:
print('Data timeline:')
print('From: ',datetime.fromtimestamp(coins['Timestamp'].iloc[0]/1000))
print('To:   ', datetime.fromtimestamp(coins['Timestamp'].iloc[-1]/1000))

Data timeline:
From:  2019-11-26 17:18:59.999000
To:    2020-02-09 02:58:59.999000


In [18]:
STEPS_IN_FUTURE = 15
PAST_DATA_SIZE = 120 #3*60
n_frames = int(len(coins)/PAST_DATA_SIZE)
n_frames

892

In [19]:
X_train =[]
y_train = []

for frame in range(0, n_frames-1):
    X_train.append([])
    y_train.append([])
    for i in range(frame, (frame+PAST_DATA_SIZE)):
        X_train[frame].append(coins.loc[i].values)
    for k in range(frame+PAST_DATA_SIZE, frame+PAST_DATA_SIZE+STEPS_IN_FUTURE):
        y_train[frame].append(coins.loc[k].values)

In [20]:
len(X_train)

891

In [21]:
X_train[0]

[array([1.45990000e+02, 1.46080000e+02, 1.45870000e+02, 1.45880000e+02,
        2.41285030e+02, 3.52184167e+04, 8.40000000e+01, 4.96838800e+01,
        7.25420911e+03, 0.00000000e+00, 7.12639000e+03, 7.13335000e+03,
        7.12301000e+03, 7.12328000e+03, 5.18538330e+01, 3.69667539e+05,
        3.69000000e+02, 3.75934310e+01, 2.68034661e+05, 0.00000000e+00,
        2.17440000e-01, 2.17600000e-01, 2.17440000e-01, 2.17510000e-01,
        1.90423000e+04, 4.14133792e+03, 3.20000000e+01, 1.23968000e+04,
        2.69588849e+03, 0.00000000e+00, 4.66400000e+01, 4.66600000e+01,
        4.65900000e+01, 4.65900000e+01, 2.82464200e+01, 1.31673559e+03,
        2.00000000e+01, 9.04073000e+00, 4.21691594e+02, 0.00000000e+00,
        1.57478514e+12]),
 array([1.45890000e+02, 1.45960000e+02, 1.45840000e+02, 1.45960000e+02,
        4.08542900e+01, 5.95946644e+03, 4.90000000e+01, 2.04204800e+01,
        2.97887129e+03, 0.00000000e+00, 7.12507000e+03, 7.12670000e+03,
        7.12176000e+03, 7.12670000e+03

In [22]:
len(y_train)

891

In [23]:
len(X_train)

891

In [24]:
X_test = np.array(X_train[-200:])
y_test = np.array(y_train[-200:])
X_train = np.array(X_train[:len(X_train)-200])
y_train = np.array(y_train[:len(y_train)-200])

In [25]:
X_train.shape

(691, 120, 41)

In [26]:
y_train.shape

(691, 15, 41)

In [27]:
X_train[0]

array([[1.45990000e+02, 1.46080000e+02, 1.45870000e+02, ...,
        4.21691594e+02, 0.00000000e+00, 1.57478514e+12],
       [1.45890000e+02, 1.45960000e+02, 1.45840000e+02, ...,
        3.39211186e+02, 0.00000000e+00, 1.57478520e+12],
       [1.45980000e+02, 1.45980000e+02, 1.45780000e+02, ...,
        8.40244063e+02, 0.00000000e+00, 1.57478526e+12],
       ...,
       [1.44080000e+02, 1.44240000e+02, 1.44070000e+02, ...,
        7.43361575e+02, 0.00000000e+00, 1.57479216e+12],
       [1.44190000e+02, 1.44330000e+02, 1.44180000e+02, ...,
        1.44884169e+03, 0.00000000e+00, 1.57479222e+12],
       [1.44300000e+02, 1.44300000e+02, 1.44100000e+02, ...,
        1.37013845e+03, 0.00000000e+00, 1.57479228e+12]])

In [28]:
y_train.shape

(691, 15, 41)

In [29]:
train_data_multi = tf.data.Dataset.from_tensor_slices((X_train, y_train))